In [1]:
import pandas as pd
from config import chromeSetup,get_selected_cols_df,ensure_dir,get_Drivers,encode_search_word,enter_sys_value,update_iod_tms_with_now,transform
import warnings
import shutil
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException
import selenium.webdriver.support.expected_conditions as EC
import time 
from datetime import datetime
from glob import glob
from tqdm.notebook import tqdm as tqdm_notebook
import json
import os
import requests
from dateutil.relativedelta import relativedelta, FR,TH
from selenium.webdriver.support.ui import Select
from collections import ChainMap
from selenium import webdriver
import numpy as np

In [2]:
timestamp = str(datetime.now())        
today_date = timestamp[:10]
log_name = os.path.join(today_date,"samedate_log.csv")
today_date


'2021-05-19'

In [3]:
### Need changed

json_path = ".\\config.json"
webdriver_path = "C:\\Users\\marco.wang\\Downloads\\chromedriver_win32\\chromedriver"

In [4]:
options = chromeSetup()

with open(json_path) as f:
    data = json.load(f)

browers = webdriver.Chrome(webdriver_path,options=options)

print("get browsers!!")
log_in_url = 'http://tms.us.lge.com/tm/framework/Frame.jsp'
browers.get(log_in_url)
time.sleep(3)
browers.switch_to.frame('i2ui_shell_content')
time.sleep(3)
browers.switch_to.frame('resultFrame')


user = browers.find_element_by_name("loginUser")
user.send_keys(data['user'])
password = browers.find_element_by_name("dspLoginPassword")
password.send_keys(data['password'])
print("log in !!")

button = browers.find_element_by_id("buttonEmphasized")
button.click()


time.sleep(3)
browers.switch_to.default_content()
browers.switch_to.frame('i2ui_shell_content')
time.sleep(3)
browers.switch_to.frame('nav')



get browsers!!
log in !!


In [5]:
browers.switch_to.default_content()
browers.switch_to.frame('i2ui_shell_content')
time.sleep(3)
browers.switch_to.frame('nav')
IOD = browers.find_element_by_xpath("//*[@id='TREECELL_navigation_18']/a")
IOD.click()
time.sleep(3)


In [6]:

status = browers.find_element_by_xpath("//*[@id='TREECELL_navigation_18_2']/a[2]")
status.click()
time.sleep(8)

In [6]:
browers.switch_to.default_content()
time.sleep(0.3)
browers.switch_to.frame('i2ui_shell_content')
time.sleep(20)
browers.switch_to.frame('resultFrame')
time.sleep(0.3)


In [7]:
js = 'document.querySelectorAll("select")[0].style.display="block";'
browers.execute_script(js)
s1 = Select(browers.find_element_by_id('searchDiv'))
s1.select_by_value("LGEUS")
js = 'document.querySelectorAll("select")[2].style.display="block";'
browers.execute_script(js)

s2 = Select(browers.find_element_by_id('searchIodCmplYn'))
s2.select_by_index(0)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchDiv"]"}
  (Session info: chrome=88.0.4324.190)


In [9]:
last_friday = (datetime.now() + relativedelta(weekday=FR(-2))).strftime("%m/%d/%Y")
last_thurday = (datetime.now() + relativedelta(weekday=TH(-1))).strftime("%m/%d/%Y")

In [10]:
searchStartEta = browers.find_element_by_id("searchStartEta")
searchStartEta.click()
print("last_friday ", last_friday)
searchStartEta.send_keys(last_friday)
searchEndEta = browers.find_element_by_id("searchEndEta")
searchEndEta.click()
print("last_thurday " ,last_thurday)
searchEndEta.send_keys(last_thurday)

last_friday  10/09/2020
last_thurday  10/15/2020


In [11]:
searchButton = browers.find_element_by_id("searchButton")
searchButton.click()

In [12]:
browers.switch_to.default_content()
time.sleep(0.3)
browers.switch_to.frame('i2ui_shell_content')
time.sleep(100)
browers.switch_to.frame('resultFrame')
time.sleep(0.3)
row = 8
while row < 20:
    total = browers.find_element_by_id(f"iodMonitoringStatusGrid_BODY_{row}_2")
    print(total.text)
    if total.text == "Total":
        print(total.text)
        break
    else: 
        row += 1
total = browers.find_element_by_xpath(f"//*[@id='iodMonitoringStatusGrid_BODY_{row}_3']/span/a")
time.sleep(30)
total.click()
print("Done!!")

TL
Total
Total
Done!!


In [13]:
driver = browers
original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break
             
files = glob("C:\\Users\\marco.wang\\Downloads\\IOD_MONITORING_DETAIL_US_*.csv")
for f in files:
    os.remove(f)
time.sleep(100)

In [14]:
wait = WebDriverWait(driver, 300, poll_frequency=10, ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException])
element = wait.until(EC.element_to_be_clickable((By.ID, "iodMonitoringDetailSaveExcel")))
        
# download = driver.find_element_by_id("iodMonitoringDetailSaveExcel")
element.click()

In [15]:

### move the data 
time.sleep(180)

In [14]:


downloaded_file = glob("C:\\Users\\marco.wang\\Downloads\\IOD_MONITORING_DETAIL_US_*.csv")[-1]
shutil.move(downloaded_file, ".\\weekly_to_data")


IndexError: list index out of range

In [3]:
SANGYEOP_CARRIER_CODE = pd.read_excel("marco_carriers.xlsx")['Carrier Code'].to_list()



In [4]:

FILE_PATH = glob(".\\weekly_to_data\\IOD_MONITORING_DETAIL_US_*.csv")[-1]
print(FILE_PATH)
df_origin = pd.read_csv(FILE_PATH)
## Columns kept
cols = ['Pro No','Load ID','Stop Seq','Carrier Code','Service Type','Sales','Ship Date',
        'Drop Appointment','IOD Report Date','Delivery Date','ETA','Customer Name']
df = df_origin[df_origin['Carrier Code'].isin(SANGYEOP_CARRIER_CODE)][cols].reset_index(drop=True)

.\weekly_to_data\IOD_MONITORING_DETAIL_US_2021419162145458260932.csv


In [5]:
df['Carrier Code'] = df['Carrier Code'].replace(['RDWY_CRA',
             'RDWY_FON',
             'RDWY_FTW',
             'RDWY_POR',
             'RDWY_ROM'],'RDWY')
df['Carrier Code'] = df['Carrier Code'].replace(['AVRT_FTW', 'AVRT', 'AVRT_ATL'],'AVRT')


In [6]:
df['Carrier Code'].unique()

array(['PGAA', 'AERG', 'CGFR', 'ISCO', 'TQLI_FW', 'AFNW', 'TFXH', 'HSXF',
       'UTP2', 'HRCF', 'RDWY', 'AVRT', 'BTVP', 'HMES', 'RETL', 'DSRE',
       'NFIL', 'CVYI', 'AFXN'], dtype=object)

In [7]:
df['Ship Date'] = df['Ship Date'].apply(pd.to_datetime)
df['Drop Appointment'] = df['Drop Appointment'].apply(pd.to_datetime)
df['IOD Report Date'] = df['IOD Report Date'].apply(pd.to_datetime)
df['Delivery Date'] = df['Delivery Date'].apply(pd.to_datetime)
df.ETA = df.ETA.apply(pd.to_datetime)


In [8]:
def get_sameday(x):
    if x.isnull()['IOD Report Date']:
        return "Pending"
    if x['IOD Report Date'].day - x['Delivery Date'].day == 0:
        return "Sameday"
    else:
        return "Overdue"

def get_Times(x):
    if x.isnull()['Delivery Date']:
        return "Pending"
    if x['Delivery Date'].hour >= 17:
        return "After Hour"
    else:
        return "Business Hour"


In [9]:
df['Sameday'] = df.apply(lambda x : get_sameday(x), axis=1)
df['WeekNum'] = df.apply(lambda x : str(x['Delivery Date'].week) if x['Delivery Date'] is not pd.NaT else "Pending", axis=1)
df['Weekend'] = df.apply(lambda x : "weekday" if x['Delivery Date'].weekday() <= 4 else "weekend", axis=1)
df['Delivery Time'] = df.apply(lambda x : x['Delivery Date'].strftime('%H:%M') if x['Delivery Date'] is not pd.NaT else "Pending", axis=1)
df['Times'] = df.apply(lambda x : get_Times(x), axis=1)
df['Pending'] = df.apply(lambda x : "Pending" if x['Delivery Date'] is pd.NaT else "Complete", axis=1)

In [10]:
## clean data for all cariers
df = df[df['Pending'] == 'Complete'].reset_index(drop=True)

In [11]:
def get_pivot_tbale(df):
    func = lambda x: 100*x.count()/df.shape[0]
    daily_report = pd.pivot_table(df,index=["Carrier Code","Weekend"],
                                  columns="Sameday",values=["Load ID"],
                                  aggfunc={
                                      "Load ID":"count",
                                  },
                                  fill_value=0,margins = True)
    daily_report = daily_report.iloc[:, :-1]
    daily_report.columns.set_levels(['# of Loads'],level = 0,inplace=True)

    daily_report1 = pd.pivot_table(df,index=["Carrier Code","Weekend"],
                                  columns="Sameday",values=["Load ID"],
                                  aggfunc={
                                      "Load ID" :func,
                                  },
                                  fill_value=0,margins = True)
    daily_report1 = daily_report1.iloc[:, :-1]
    daily_report1.columns.set_levels(['% of Loads'],level = 0,inplace=True)
    daily_report_a = pd.concat([daily_report,daily_report1],axis = 1)


    daily_report = pd.pivot_table(df,index=["Carrier Code","Sameday"],
                                  columns=["Weekend","Times"],values=["Load ID"],
                                  aggfunc={
                                      "Load ID":"count",
                                  },
                                  fill_value=0, margins = True)
    daily_report_b = daily_report.iloc[:, :-1]
    return daily_report_a, daily_report_b

In [12]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

weekstamp = str(datetime.now().isocalendar()[1])
date_path = os.path.join('weekly_report',weekstamp)
ensure_dir(date_path)

writer = pd.ExcelWriter(os.path.join(date_path,'weekly_performance_to_carriers.xlsx'), engine='xlsxwriter')
workbook=writer.book
# Write each dataframe to a different worksheet. you could write different string like above if you want

for carrier_code in SANGYEOP_CARRIER_CODE:
    df_ONE_CARRIERS = df[df['Carrier Code'] == carrier_code]
    if df_ONE_CARRIERS.shape[0] == 0:
        continue
    print(f"Working on {carrier_code} !!!!!")
    daily_report_a, daily_report_b = get_pivot_tbale(df_ONE_CARRIERS)
    worksheet=workbook.add_worksheet(carrier_code)
    writer.sheets[carrier_code] = worksheet
    daily_report_a.to_excel(writer,sheet_name=carrier_code,startrow=1 , startcol=0)
    worksheet.write_string(daily_report_a.shape[0] + 8, 0, "total")
    daily_report_b.to_excel(writer,sheet_name=carrier_code,startrow=daily_report_a.shape[0] + 5, startcol=0)
    worksheet.write_string(daily_report_a.shape[0] + daily_report_b.shape[0] + 8, 0, "total")
    df[df ['Carrier Code'] == carrier_code][df ['Sameday'] == "Overdue"].to_excel(writer,sheet_name=carrier_code,startrow=daily_report_a.shape[0] + daily_report_b.shape[0] + 12, startcol=0,index= False)

writer.save()

Working on AVRT !!!!!


C:\Users\marco.wang\Miniconda3\envs\pantos\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Working on BTVP !!!!!
Working on RDWY !!!!!
Working on TQLI_FW !!!!!
Working on TFXH !!!!!
Working on HMES !!!!!
Working on RETL !!!!!
Working on HRCF !!!!!
Working on UTP2 !!!!!
Working on NFIL !!!!!
Working on AFXN !!!!!
Working on AFNW !!!!!
Working on ISCO !!!!!
Working on CVYI !!!!!
Working on PGAA !!!!!
Working on AERG !!!!!
Working on HSXF !!!!!
Working on CGFR !!!!!
Working on DSRE !!!!!
